In [1]:
import imp
import pandas as pd
import nltk
import re
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression
from math import sqrt

C:\Users\nel_1\AppData\Local\Temp\ipykernel_15396\1314585710.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
true_df = pd.read_csv('./True.csv')
fake_df = pd.read_csv('./Fake.csv')
true_df['label'] = 0
fake_df['label'] = 1
true_df = true_df[['text','label']]
fake_df = fake_df[['text','label']]
dataset = pd.concat([true_df , fake_df])
print(dataset['label'].value_counts())

1    23481
0    21417
Name: label, dtype: int64


In [3]:
dataset = dataset.sample(frac = 1)
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))
def clean_data(text):
    text = text.lower() 
    text = re.sub('[^a-zA-Z]' , ' ' , text)
    token = text.split() 
    news = [lemmatizer.lemmatize(word) for word in token if not word in stopwords]  
    clean_news = ' '.join(news) 
    
    return clean_news 

In [4]:
dataset['text'] = dataset['text'].apply(lambda x : clean_data(x))
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))
X = dataset.iloc[:2000,0]
y = dataset.iloc[:2000,1]
print(X.head())
print(y.head())

15518    american need put american citizen first dump ...
6330     washington reuters president elect donald trum...
18845    refugee ghana arrested dragging young woman te...
11156    border patrol council president brandon judd v...
8644     ted nugent woke wrong side bed wednesday morni...
Name: text, dtype: object
15518    1
6330     0
18845    1
11156    1
8644     1
Name: label, dtype: int64


In [5]:
train_X , test_X , train_y , test_y = train_test_split(X , y , test_size = 0.9 ,random_state = 0)
vec_train = vectorizer.fit_transform(train_X)
vec_train = vec_train.toarray()
vec_test = vectorizer.transform(test_X).toarray()
train_data = pd.DataFrame(vec_train , columns=vectorizer.get_feature_names_out())
test_data = pd.DataFrame(vec_test , columns= vectorizer.get_feature_names_out())
clf = LogisticRegression()
clf.fit(train_data, train_y)
predictions = clf.predict(test_data)
print(classification_report(test_y , predictions))


              precision    recall  f1-score   support

           0       0.95      0.91      0.93       864
           1       0.92      0.95      0.94       936

    accuracy                           0.93      1800
   macro avg       0.93      0.93      0.93      1800
weighted avg       0.93      0.93      0.93      1800

